In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
model = load_model("model_final.keras")

In [3]:
class_labels = {0: 'Feliz', 1: 'Triste', 2: 'Surpreso'}

In [ ]:
# Iniciar a webcam
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()

    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostos no frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (48, 48)) 

        # Normalizar a imagem para a escala [0, 1]
        face_normalized = face_resized / 255.0

        face_expanded = np.expand_dims(face_normalized, axis=0)
        face_expanded = np.expand_dims(face_expanded, axis=-1)

        prediction = model.predict(face_expanded)
        predicted_label = np.argmax(prediction)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        emotion_label = class_labels[predicted_label]
        cv2.putText(frame, emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow('Reconhecimento de Emoção', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step


In [8]:
import cv2
import numpy as np

# Carregar o vídeo em vez da webcam
cap = cv2.VideoCapture("emotional_dagame.mp4")

# Carregar o classificador de faces
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter('video_com_deteccoes.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostos no frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (48, 48)) 

        face_normalized = face_resized / 255.0

        face_expanded = np.expand_dims(face_normalized, axis=0)
        face_expanded = np.expand_dims(face_expanded, axis=-1)

        prediction = model.predict(face_expanded)
        predicted_label = np.argmax(prediction)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        emotion_label = class_labels[predicted_label]
        cv2.putText(frame, emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)


    out.write(frame)
    cv2.imshow('Reconhecimento de Emoção', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 18ms/step
